# Clean Masjid
Clean masjid dataset from QuickOSM.

In [68]:
# import 
import os 
import geopandas as gpd 

In [69]:
# read geodata
osm_masjid_gdf = gpd.read_file('./raw/osm_masjid.geojson')

display(osm_masjid_gdf.sample())
display(osm_masjid_gdf.shape)

,full_id,osm_id,osm_type,religion,amenity,source_date,name:en,survey:date,addr:province,addr:district,...,unmil:id,wheelchair,name:fr,building,addr:state,addr:country,addr:city,name,denomination,geometry
897,n7643528933,7643528933,node,muslim,place_of_worship,NaT,None,NaT,None,None,...,None,None,None,None,None,None,None,None,None,POINT (-7.49085 6.55148)


(1181, 52)

In [70]:
# keep cols
keep_col = ['full_id', 'name', 'denomination','geometry']
osm_masjid_gdf = osm_masjid_gdf[keep_col]

# rename cols
osm_masjid_gdf = osm_masjid_gdf.rename(columns={'full_id': 'osm_id', 'denomination': 'category'})

# drop duplicates
osm_masjid_gdf = osm_masjid_gdf.drop_duplicates(subset=['osm_id'])

# add country
osm_masjid_gdf['country'] = "Cote d'Ivoire"
osm_masjid_gdf['source'] = 'osm'

display(osm_masjid_gdf)

,osm_id,name,category,geometry,country,source
0,n715289027,None,None,POINT (-3.24677 10.96462),Cote d'Ivoire,osm
1,n768587386,Mosquée Aicha Niangon Sud de Yopougon,sunni,POINT (-4.08853 5.32643),Cote d'Ivoire,osm
2,n775206010,None,None,POINT (-3.23219 10.96773),Cote d'Ivoire,osm
3,n775206016,None,None,POINT (-3.24442 10.96401),Cote d'Ivoire,osm
4,n844756893,Mosquée Libanaise de Marcory,None,POINT (-3.99165 5.30388),Cote d'Ivoire,osm
...,...,...,...,...,...,...
1176,n13208374055,Fatima Abdullah Khalifa Al-Badur Mosque,None,POINT (-2.24621 9.85747),Cote d'Ivoire,osm
1177,n13279073742,None,None,POINT (-2.44776 9.73346),Cote d'Ivoire,osm
1178,n13279611452,None,None,POINT (-2.41942 9.49541),Cote d'Ivoire,osm
1179,n13279616863,None,None,POINT (-2.48426 9.52571),Cote d'Ivoire,osm


In [71]:
# clip on CIV
civ_gdf = gpd.read_file('./raw/abidjan.gpkg')
osm_masjid_gdf = gpd.clip(osm_masjid_gdf, civ_gdf)
osm_masjid_gdf = osm_masjid_gdf.reset_index(drop=True)

display(osm_masjid_gdf.sample())
display(osm_masjid_gdf.shape)

,osm_id,name,category,geometry,country,source
59,n10595935677,Après le pont,None,POINT (-4.07337 5.33639),Cote d'Ivoire,osm


(374, 6)

In [72]:
# add image url
osm_masjid_gdf['image_url'] = 'https://unsplash.com/fr/photos/un-grand-batiment-avec-deux-tours-et-un-dome-zBn742lw6Bk'

display(osm_masjid_gdf.sample())

,osm_id,name,category,geometry,country,source,image_url
4,n7142652607,None,None,POINT (-4.09255 5.34199),Cote d'Ivoire,osm,https://unsplash.com/fr/photos/un-grand-batime...


In [73]:
# save 
osm_masjid_gdf.to_file('./temp/osm_masjid-clean-20251109.geojson', driver='GeoJSON')

In [74]:
# generate lat & long
osm_masjid_gdf['latitude'] = osm_masjid_gdf['geometry'].apply(
    lambda geom: geom.y
)
osm_masjid_gdf['longitude'] = osm_masjid_gdf['geometry'].apply(
    lambda geom: geom.x
)

In [75]:
# keep col
keep_col = ['name', 'category', 'country', 'latitude', 'longitude', 'image_url']
osm_masjid_df = osm_masjid_gdf[keep_col]

osm_masjid_df.to_csv("./temp/osm_masjid-clean-20251109.csv", index=False)

In [76]:
osm_masjid_df

,name,category,country,latitude,longitude,image_url
0,Mosquée Darou-Salam,None,Cote d'Ivoire,5.339633,-4.084064,https://unsplash.com/fr/photos/un-grand-batime...
1,Mosquée Du Maroc,None,Cote d'Ivoire,5.340382,-4.097596,https://unsplash.com/fr/photos/un-grand-batime...
2,None,None,Cote d'Ivoire,5.340913,-4.104043,https://unsplash.com/fr/photos/un-grand-batime...
3,None,None,Cote d'Ivoire,5.341426,-4.106926,https://unsplash.com/fr/photos/un-grand-batime...
4,None,None,Cote d'Ivoire,5.341989,-4.092547,https://unsplash.com/fr/photos/un-grand-batime...
...,...,...,...,...,...,...
369,None,None,Cote d'Ivoire,5.455064,-4.046178,https://unsplash.com/fr/photos/un-grand-batime...
370,Mosquée Faridaws,None,Cote d'Ivoire,5.455222,-4.042224,https://unsplash.com/fr/photos/un-grand-batime...
371,Mosquée Akira Bismirabika,None,Cote d'Ivoire,5.456527,-4.049442,https://unsplash.com/fr/photos/un-grand-batime...
372,Mosquée Sanogo,None,Cote d'Ivoire,5.460830,-4.049512,https://unsplash.com/fr/photos/un-grand-batime...
